In [ ]:
!pip install snowflake-connector-python
!pip install snowflake-connector-python[pandas]

In [8]:
import pandas as pd

df = pd.read_csv("data/ClinicalTrialsData.csv",index_col="Unnamed: 0")
df.head()

,nctid,target_label,number_collaborators,number_officials,num_arms,number_countries,number_sites,no_elig_req,inclusion_lines,exclusion_lines,...,90,91,92,93,94,95,96,97,98,99
0,NCT01923116,0,2,2,1.0,1,1,0,6,6,...,3.039853,-3.300040,1.487360,1.478261,2.550636,1.517649,1.237359,3.249761,1.415775,1.477161
1,NCT01763021,0,0,1,1.0,1,1,0,2,4,...,0.513191,-1.393093,0.940864,0.088683,0.074904,-1.793713,0.749069,1.982955,-1.104483,1.344036
2,NCT01179308,1,0,1,2.0,2,2,0,3,4,...,2.163420,-0.411902,-0.035816,3.557102,-2.521656,-3.917873,0.496561,1.281240,-0.059275,-0.031860
3,NCT00427778,1,1,1,2.0,1,1,0,3,3,...,3.769808,-1.176595,0.247578,1.944952,-0.830502,0.593395,-2.722616,0.866244,2.003813,1.861063
4,NCT00376467,0,0,1,0.0,1,28,0,3,9,...,-2.228156,-4.341137,1.126694,2.487203,1.253972,-0.105168,3.125254,3.700899,1.906987,3.855995


In [3]:
data_types = df.dtypes
data_types

Unnamed: 0                int64
nctid                    object
target_label              int64
number_collaborators      int64
number_officials          int64
                         ...   
95                      float64
96                      float64
97                      float64
98                      float64
99                      float64
Length: 643, dtype: object

In [9]:
# Get data types of the columns in the dataframe
data_types = df.dtypes

# Initialize an empty list to hold column definitions
column_definitions = []

# Mapping from pandas data types to Snowflake data types
type_mapping = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'TEXT',     
}

# Generate the column definitions for the Snowflake table
for column_name, data_type in data_types.items():
    snowflake_type = type_mapping.get(str(data_type), 'TEXT')
    column_definitions.append(f'"{column_name}" {snowflake_type}')


In [10]:
column_definitions

['"nctid" TEXT',
 '"target_label" INTEGER',
 '"number_collaborators" INTEGER',
 '"number_officials" INTEGER',
 '"num_arms" FLOAT',
 '"number_countries" INTEGER',
 '"number_sites" INTEGER',
 '"no_elig_req" INTEGER',
 '"inclusion_lines" INTEGER',
 '"exclusion_lines" INTEGER',
 '"eligibility_lines" INTEGER',
 '"inclusion_words" INTEGER',
 '"exclusion_words" INTEGER',
 '"eligibility_words" INTEGER',
 '"inclusion_numbers" INTEGER',
 '"exclusion_numbers" INTEGER',
 '"eligibility_numbers" INTEGER',
 '"avg_ex_line_words" FLOAT',
 '"avg_in_line_words" FLOAT',
 '"avg_elig_words" FLOAT',
 '"has_expanded_access" INTEGER',
 '"industry_sponsor" INTEGER',
 '"industry_collaborator" INTEGER',
 '"has_dmc" INTEGER',
 '"fda_reg" INTEGER',
 '"has_oversight" INTEGER',
 '"gender_restriction" INTEGER',
 '"random_groups" INTEGER',
 '"no_phase" INTEGER',
 '"phase_1" INTEGER',
 '"phase_2" INTEGER',
 '"phase_3" INTEGER',
 '"phase_4" INTEGER',
 '"responsible_party_category_none" INTEGER',
 '"responsible_party_cate

In [12]:
# Join all column definitions in a single string separated by commas
column_definitions_str = ",\n    ".join(column_definitions)
column_definitions_str

'"nctid" TEXT,\n    "target_label" INTEGER,\n    "number_collaborators" INTEGER,\n    "number_officials" INTEGER,\n    "num_arms" FLOAT,\n    "number_countries" INTEGER,\n    "number_sites" INTEGER,\n    "no_elig_req" INTEGER,\n    "inclusion_lines" INTEGER,\n    "exclusion_lines" INTEGER,\n    "eligibility_lines" INTEGER,\n    "inclusion_words" INTEGER,\n    "exclusion_words" INTEGER,\n    "eligibility_words" INTEGER,\n    "inclusion_numbers" INTEGER,\n    "exclusion_numbers" INTEGER,\n    "eligibility_numbers" INTEGER,\n    "avg_ex_line_words" FLOAT,\n    "avg_in_line_words" FLOAT,\n    "avg_elig_words" FLOAT,\n    "has_expanded_access" INTEGER,\n    "industry_sponsor" INTEGER,\n    "industry_collaborator" INTEGER,\n    "has_dmc" INTEGER,\n    "fda_reg" INTEGER,\n    "has_oversight" INTEGER,\n    "gender_restriction" INTEGER,\n    "random_groups" INTEGER,\n    "no_phase" INTEGER,\n    "phase_1" INTEGER,\n    "phase_2" INTEGER,\n    "phase_3" INTEGER,\n    "phase_4" INTEGER,\n    "res

In [13]:
import snowflake.connector

USER = "<SNOWFLAKE_USER>"
PASSWORD = "<SNOWFLAKE_PASSWORD>"
ACCOUNT = "<SNOWFLAKE_ACCOUNT>"

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,         
)

# Create a cursor object
cur = conn.cursor()

try:

    # Create a new warehouse
    cur.execute("CREATE WAREHOUSE IF NOT EXISTS clinical_trials_warehouse")

    # Create a new database
    cur.execute("CREATE DATABASE IF NOT EXISTS clinical_trials_db")

    # Use the newly created database and warehouse
    cur.execute("USE WAREHOUSE clinical_trials_warehouse")
    cur.execute("USE DATABASE clinical_trials_db")

    # Create a new schema
    cur.execute("CREATE SCHEMA IF NOT EXISTS clinical_trials_schema")

    # Use the newly created schema
    cur.execute("USE SCHEMA clinical_trials_schema")

    # Create a new table with the column definitions
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS clinical_trial (
        {column_definitions_str}
    )
    """
    cur.execute(create_table_sql)

finally:
    # Close the cursos and the connection
    cur.close()
    conn.close()

In [17]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse='clinical_trials_warehouse',
    database='clinical_trials_db',
    schema='clinical_trials_schema'
)

# Create a cursor object
cur = conn.cursor()

try:

    # Set the schema
    cur.execute("USE SCHEMA clinical_trials_db.clinical_trials_schema")

    # Write data from the pandas DataFrame to the Snowflake table
    success, nchunks, nrows, _ = write_pandas(conn, df, 'CLINICAL_TRIAL')

    print(f"Successfully inserted {nrows} rows into the table.")

finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()

C:\Users\m1000\AppData\Local\Temp\ipykernel_38288\1964639291.py:23: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, df, 'CLINICAL_TRIAL')


Successfully inserted 68999 rows into the table.
